In [ ]:
import bt
import backtesting
import pandas as pd
from tqdm import tqdm
import numpy as np

### 1. Upload Data

In [9]:
data_path = '/Users/ljhee/Desktop/BOAZ/Fintshift/Data/'

price_stable_final = pd.read_csv(f'{data_path}안정성_주가_백테스트용.csv')
price_value_final = pd.read_csv(f'{data_path}가치성_주가_백테스트용.csv')
price_grow_final = pd.read_csv(f'{data_path}성장성_주가_백테스트용.csv')
price_cost_final = pd.read_csv(f'{data_path}수익성_주가_백테스트용.csv')

In [ ]:
df = pd.concat([price_stable_final,price_value_final,price_grow_final,price_cost_final],axis=1)

In [11]:
cost = pd.read_csv(f"{data_path}15_23_주가데이터_1201.csv")

cost['stock_code'] =  [str(num).zfill(6) for num in cost['stock_code']]

#### 주가 데이터 16년도부터로 변경
date = cost['Date'].str.split('-')
idx=[]
for i,d in enumerate(date):
     if d[0] != '2015':
        idx.append(i)

cost = cost.iloc[idx,:]

/var/folders/lf/cj47lqmj29jccg4zzhvws_tr0000gn/T/ipykernel_52901/706449337.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  cost = pd.read_csv(f"{data_path}15_23_주가데이터_1201.csv")


### 2. Buy and Hold

In [ ]:
strategy = bt.Strategy("Asset_EW", [
    bt.algos.SelectAll(),
    bt.algos.WeighEqually(),
    bt.algos.RunOnce(),####buy and hold
    bt.algos.Rebalance()
])

bnh_df = pd.DataFrame()

for _col in df.columns:
    data = df[[_col]].fillna(0)
    data.index = cost[cost['stock_code']=='071670']['Date']
    data.index = pd.to_datetime(data.index)
    # 백테스트 생성
    backtest = bt.Backtest(strategy, data)

    # 백테스트 실행
    result = bt.run(backtest)

    ndf = result.prices.to_returns()
    ndf.columns = [_col]
    bnh_df = pd.concat([bnh_df,ndf],axis = 1)

### 3. Mean Reversion Strategy

In [ ]:
reg_df = pd.DataFrame()

for _col in tqdm(df.columns):
    data = df[[_col]]
    data.index = cost[cost['stock_code']=='071670']['Date']
    data.index = pd.to_datetime(data.index)
    
    spy_rsi = data.apply(lambda x: talib.RSI(x, 14))

    signal = spy_rsi.copy()
    signal[spy_rsi > 70] = -1
    signal[spy_rsi < 30] = 1
    signal[(spy_rsi <= 70) & (spy_rsi >= 30)] = 0
    signal[signal.isnull()] = 0
    
    strategy = bt.Strategy('RSI_MeanReversion',
                       [bt.algos.WeighTarget(signal),
                        bt.algos.Rebalance()])
    
    backtest = bt.Backtest(strategy, data)
    result = bt.run(backtest)
    
    ndf = result.prices.to_returns()
    ndf.columns = [_col]
    reg_df = pd.concat([reg_df,ndf],axis = 1)

In [ ]:
reg_df.to_csv(f'{data_path}개별주식_평균회귀.csv',index=False)

### 4. Moving Average Crossover

In [ ]:
class SelectWhere(bt.Algo):

    """
    Selects securities based on an indicator DataFrame.

    Selects securities where the value is True on the current date (target.now).

    Args:
        * signal (DataFrame): DataFrame containing the signal (boolean DataFrame)

    Sets:
        * selected

    """
    def __init__(self, signal):
        self.signal = signal

    def __call__(self, target):
        # get signal on target.now
        if target.now in self.signal.index:
            sig = self.signal.loc[target.now]

            # get indices where true as list
            selected = list(sig.index[sig])

            # save in temp - this will be used by the weighing algo
            target.temp['selected'] = selected

        # return True because we want to keep on moving down the stack
        return True

In [ ]:
MAC_df = pd.DataFrame()

for _col in tqdm(df.columns):
    data = df[[_col]].fillna(0)
    data.index = cost[cost['stock_code']=='071670']['Date']
    data.index = pd.to_datetime(data.index)

    sma = data.rolling(50).mean()

    signal = data > sma
    # 백테스트 생성

    MAC = bt.Strategy('above50sma', [SelectWhere(data > sma),
                               bt.algos.WeighEqually(),
                               bt.algos.Rebalance()])
    
    backtest = bt.Backtest(MAC, data)

    # 백테스트 실행
    result = bt.run(backtest)

    ndf = result.prices.to_returns()
    ndf.columns = [_col]
    MAC_df = pd.concat([MAC_df,ndf],axis = 1)

### 5. Data for three strategies (Buy and hold, Mean Reversion Stragey, Moving Average Crossover)

In [ ]:
stock = pd.read_excel(f'{data_path}2_dartdata_variable_1222.xlsx')
stock = stock[['종목코드','종목명']]
stock.drop_duplicates(inplace=True)
stock_dic = dict(zip(stock['종목코드'], stock['종목명']))
reg_df['variable'].map(stock_dic)

In [ ]:
reg_df = pd.read_csv(f'{data_path}개별주식_평균회귀.csv')
mac_df = pd.read_csv(f'{data_path}개별주식_동적평균거래.csv')
bh = pd.read_csv(f'{data_path}개별주식_buy_and_hold.csv')

In [ ]:
mac_df = mac_df.melt()
mac_df['투자전략'] = '평균회귀'
mac_df['time'] = list(ndf.index)*2430
mac_df['variable'] = mac_df['variable'].map(stock_dic)
reg_df = reg_df.melt()
reg_df['투자전략'] = '동적평균거래'
reg_df['time'] = list(ndf.index)*2430
reg_df['variable'] = reg_df['variable'].map(stock_dic)
bh = bh.melt()
bh['투자전략'] = 'buy and hold'
bh['time'] = list(ndf.index)*2430
bh['variable'] = bh['variable'].map(stock_dic)

In [ ]:
final = pd.concat([mac_df,reg_df,bh],axis=0)
final.columns = ['종목명','수익률','투자전략','시점']
final.to_csv(f'{data_path}개별주식_투자전략3개.csv',index=False)